<h2><center>Методы анализа текстов</center></h2>

### Этапы анализа текстов

При обработке текстов выделяют несколько этапов анализа.
* Токенизация (графематический анализ) - выделение абзацев, предложений, токенов. Если абзацы в HTML выделяются довольно просто - по тегам &lt;p&gt;, то с выделением предложений и слов могут быть проблемы.
`Г. Мурманск был основан 3 апреля 1915 г. ниже впадения р. Туломы в Кольский залив. Минимальные IP-адреса: 109.124.97.0 - 109.124.97.3.`
* Морфологический анализ (стемминг, лемматизация) - определение начальной формы слова или его псевдопрефикса, грамматических параметров. Подробнее описан ниже.
* Синтаксический анализ - определение связей между словами (деревья зависимостей) или синтаксически связанных групп слов (деревья составляющих). Первые больше подходят для русского языка, вторые - для английского.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/0/0d/Wearetryingtounderstandthedifference_%282%29.jpg">
    <a href="https://en.wikipedia.org/wiki/Dependency_grammar">Деревья зависимостей</a>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Competing_sentence_diagrams.png/750px-Competing_sentence_diagrams.png">
<a href="https://en.wikipedia.org/wiki/Constituent_(linguistics)">Деревья составляющих</a></center>

* Семантический анализ - определение смысла слова и работа с ним (`за'мок` vs `замо'к`, `удаление ребра связанного графа`; не путать с `он видел их семью своими глазами` где имеет место грамматическая неоднозначность `семья`-`семь`). Последнее время чаще используются дистрибутивные модели языка, о которых мы поговорим на отдельном занятии.

Задачей морфологического анализа является определение начальной формы слова, его части речи и грамматических параметров. В некоторых случаях от слова требуется только начальная форма, в других - только начальная форма и часть речи.<br>
Существует два больших подхода к морфологическому анализу: <b>стемминг</b> и <b>поиск по словарю</b>. Для проведения стемминга оставляется справочник всех окончаний для данного языка. Для пришедшего слова проверяется его окончание и по нему делается прогноз начальной формы и части речи.<br>
Например, мы создаем справочник, в котором записываем все окончания прилагательных: <i>-ому, -ему, -ой, -ая, -ий, -ый, ...</i> Теперь все слова, которые имеют такое окончание будут считаться прилагаельными: <i>синий, циклический, красного, больному</i>. Заодно прилагательными будут считаться причастия (<i>делающий, строившему</i>) и местоимения (<i>мой, твой, твоему</i>). Также не понятно что делать со словами, имеющими пустое окончание. Отдельную проблему составляют такие слова, как <i>стекло, больной, вина</i>, которые могут разбираться несколькими вариантами (это явление называется <b>омонимией</b>). Помимо этого, стеммер может просто откусывать окончания, оставляя лишь псевдооснову.<br>
Большинство проблем здесь решается, но точность работы бессловарных стеммеров находится на уровне 80%. Чтобы повысить точность испольуют морфологический анализ со словарем. Разработчики составляют словарь слов, встретившихся в текстах (<a href="http://opencorpora.org/dict.php">здесь</a> можно найти пример такого словаря). Теперь каждое слово будет искаться в словаре и не предсказываться, а выдаваться точно. Для слов, отсутствующих в словаре, может применяться предсказание, пообное работе стеммера.<br>

Пусть к нам в руки попал файл с новостями с сайта http://lenta.ru/ . Нам любопытно посмотреть какую-то статистику по этому сайту и его новостям.

Файл представляет собой просто текст с некоторым форматированием. Все новости отделены друг от друга пятью знаками равно, дальше идет дата новости, пять минусов, текст новости. 

Для начала загрузим новости в DataFrame.

## Морфологический анализ

Есть несколько наиболее распространенных библиотеки для морфологического анализа текстов на Python: pymorphy2, pymystem и nltk. Рассмотрим работу с ними.

Библиотеки pymorphy основана на словаре [OpenCorpora](opencorpora.org/) и позволяет проводить анализ отдельных слов, то есть предварительно необходимо провести графематический анализ.

In [17]:
import pymorphy2 # Морфологический анализатор.

Посмотрим на результаты анализа отдельного слова.

In [40]:
morph=pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.
wordform=morph.parse('стекло')  # Проведем анализ слова "стекло"...
print(wordform)                 # ... и посмотрим на результат.

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.75, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 0),)), Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=0.1875, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 3),)), Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.0625, methods_stack=((<DictionaryAnalyzer>, 'стекло', 968, 3),))]


Как видно из вывода, слово "стекло" может быть неодушевленным существительным среднего рода, единственного числа, именительного падежа `tag=OpencorporaTag('NOUN,inan,neut sing,nomn')`, аналогично, но в винительном падеже (`'NOUN,inan,neut sing,accs'`), и глаголом `'VERB,perf,intr neut,sing,past,indc'`. При этом в первой форме оно встречается в 75% случаев (<i>score=0.75</i>), во второй в 18,75% случаев (<i>score=0.1875</i>), а как глагол - лишь в 6,25% (<i>score=0.0625</i>). Самым простым видом борьбы с омонимией является выбор нулевого элемента из списка, возвращенного морфологическим анализом. Такой подход дает около 90% точности при выборе начальной формы и до 80% если мы обращаем внимание на грамматические параметры.

In [19]:
wordform[1].score

0.1875

In [20]:
wordform[0].tag

OpencorporaTag('NOUN,inan,neut sing,nomn')

Pymorphy2 умеет синтезировать нужные нам формы слова. Для этого необходимо получить объект типа `Parse` для нужного слова, а затем вызвать функцию `inflect`.

In [41]:
wordform[0].inflect({'plur','datv'})

Parse(word='стёклам', tag=OpencorporaTag('NOUN,inan,neut plur,datv'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стёклам', 545, 8),))

Помимо этого можно получить вообще всю лексему.

In [42]:
wordform[0].lexeme

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 0),)),
 Parse(word='стекла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стекла', 545, 1),)),
 Parse(word='стеклу', tag=OpencorporaTag('NOUN,inan,neut sing,datv'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стеклу', 545, 2),)),
 Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 3),)),
 Parse(word='стеклом', tag=OpencorporaTag('NOUN,inan,neut sing,ablt'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стеклом', 545, 4),)),
 Parse(word='стекле', tag=OpencorporaTag('NOUN,inan,neut sing,loct'), normal_form='стекло', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стекле', 545, 5),)),
 Parse(word='стёкла'

Обратите внимание на `methods_stack=((<DictionaryAnalyzer>, 'лось', 121, 0)` и `methods_stack=((<FakeDictionary>, 'варкалось', 224, 9)`. Наличие строки <FakeDictionary> говорито том, что слово было предсказано. Причем в первом случае оно предсказано как форма слова _лось_, к которому добавлена неизвестная приставка. Во втором случае - это совершенно незнакомое слово.

А ещё pymorphy умеет предсказывать незнакомые слова.

In [13]:
wordform=morph.parse('пырялись') 
wordform

[Parse(word='пырялись', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='пыряться', score=1.0, methods_stack=((<FakeDictionary>, 'пырялись', 224, 10), (<KnownSuffixAnalyzer>, 'ялись')))]

Вместо Pymorphy можно использовать PyMystem. Его плюсом является тот факт, что он сам проводит графематический анализ и снимает омонимию. 

Функция `lemmatize` делит текст на слова и знаки препинания, а затем возвращает для них только начальную форму.

Функция `analyze` возвращает не только начальную форму, но и всю информацию о слове, как это делал перед этим Pymorphy. 

Как видно из примера, делает он это не всегда корректно, но нам не придется думать о том, какое вариант разбора следует взять.

In [21]:
import pymystem3 # Еще один морфологический анализатор. При первом запуске грузит словари из Сети.

In [22]:
mystem=pymystem3.Mystem()
print(mystem.lemmatize('эти типы стали есть в цеху.'))
print(mystem.analyze('эти типы стали есть в цеху.'))

['этот', ' ', 'тип', ' ', 'становиться', ' ', 'есть', ' ', 'в', ' ', 'цех', '.', '\n']
[{'analysis': [{'lex': 'этот', 'wt': 1, 'gr': 'APRO=(им,мн|вин,мн,неод)'}], 'text': 'эти'}, {'text': ' '}, {'analysis': [{'lex': 'тип', 'wt': 0.8700298667, 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 'text': 'типы'}, {'text': ' '}, {'analysis': [{'lex': 'становиться', 'wt': 0.9821285009, 'gr': 'V,нп=прош,мн,изъяв,сов'}], 'text': 'стали'}, {'text': ' '}, {'analysis': [{'lex': 'есть', 'wt': 0.04922361672, 'gr': 'V,несов,пе=инф'}], 'text': 'есть'}, {'text': ' '}, {'analysis': [{'lex': 'в', 'wt': 0.9999917746, 'gr': 'PR='}], 'text': 'в'}, {'text': ' '}, {'analysis': [{'lex': 'цех', 'wt': 1, 'gr': 'S,муж,неод=(дат,ед|местн,ед)'}], 'text': 'цеху'}, {'text': '.'}, {'text': '\n'}]


Результаты работы можно представить в таком виде.

```
['этот', 
 ' ',
 'тип',
 ' ',
 'становиться',
 ' ',
 'есть',
 ' ',
 'в',
 ' ',
 'цех',
 '.',
 '\n']```
 
```
[
  {'analysis': 
    [{'lex': 'этот', 'wt': 1, 'gr': 'APRO=(им,мн|вин,мн,неод)'}], 
   'text': 'эти'
  }, 
  {
    'text': ' '
  }, 
  {'analysis': 
    [{'lex': 'тип', 'wt': 0.8700298667, 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 
   'text': 'типы'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'становиться', 'wt': 0.9821285009, 'gr': 'V,нп=прош,мн,изъяв,сов'}], 
   'text': 'стали'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'есть', 'wt': 0.04922361672, 'gr': 'V,несов,пе=инф'}], 
   'text': 'есть'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'в', 'wt': 0.9999917746, 'gr': 'PR='}], 
   'text': 'в'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'цех', 'wt': 1, 'gr': 'S,муж,неод=(дат,ед|местн,ед)'}], 
   'text': 'цеху'}, 
  {'text': '.'}, 
  {'text': '\n'}
]
```

То есть результатом является список токенов (в том числе и пробельных или знаков препинания), для части из которых имеется результат анализа, который хранится в словаре с ключём `analysis`. Анализ хранит в списке один или несколько вариантов разбора, у каждого из которых есть лемма `lex`, набор грамматических параметров `gr` и некоторый вес `wt`, который показывает степень уверенности системы в правильности ответа.

In [6]:
my_res=mystem.analyze('эти типы стали есть в цеху.')
if 'analysis' in my_res[0].keys(): # Проверяем, что это не разделитель.
    print(my_res[0]['analysis'][0]['gr'].split("=")[0]) # Берем из него анализ, из того грамматическсие параметы, 
                                                        # а из них выделяем часть речи.

APRO


Еще одна библиотека - NLTK. По сравнению с двумя предыдущими библиотеками она обладает более широкой функциональностью и изначально писалась для работы с разными языками.

In [23]:
import nltk # Иностранный морфологический анализатор.

Перед началом использования необходимо загрузить необходимые библиотеки или корпуса.

In [32]:
nltk.download() # По дороге будут появляться поле ввода. Грузит всё из Сети.

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> l

Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [*] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [ ] cess_esp............ CESS-ESP Treebank
  [ ] chat80.....

        /home/edward/nltk_data...
      Unzipping taggers/averaged_perceptron_tagger.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

Можно сразу скачать нужный пакет, если вы знаете как он назыввается.

In [35]:
nltk.download(['averaged_perceptron_tagger_ru', 'stopwords'])

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/edward/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/edward/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Функция `word_tokenize` возвращает начальные формы слов. 

Функция `pos_tag` возвращает список начальных форм и их частей речи.

In [30]:
tokens = nltk.word_tokenize('Эти типы стали есть в цеху') # Токенизация.
bi_tokens = list(nltk.bigrams(tokens))
tokens, bi_tokens

(['Эти', 'типы', 'стали', 'есть', 'в', 'цеху'],
 [('Эти', 'типы'),
  ('типы', 'стали'),
  ('стали', 'есть'),
  ('есть', 'в'),
  ('в', 'цеху')])

In [33]:
pos = nltk.pos_tag(tokens) # Частеречная разметка.
bi_pos = list(nltk.bigrams(pos))
pos, bi_pos


([('Эти', 'JJ'),
  ('типы', 'NNP'),
  ('стали', 'NNP'),
  ('есть', 'NNP'),
  ('в', 'NNP'),
  ('цеху', 'NN')],
 [(('Эти', 'JJ'), ('типы', 'NNP')),
  (('типы', 'NNP'), ('стали', 'NNP')),
  (('стали', 'NNP'), ('есть', 'NNP')),
  (('есть', 'NNP'), ('в', 'NNP')),
  (('в', 'NNP'), ('цеху', 'NN'))])

У NLTK заведен список стоп-слов, которые лучше фильтровать при анализе текстов. Но их не очень много. Зато самые мешающиеся.

In [36]:
# Оставим только те слова, которых нет в списке стоп-слов.
filtered_words = [token for token in tokens if token not in nltk.corpus.stopwords.words('russian')]
print('всего русских стоп-слов', len(nltk.corpus.stopwords.words('russian')))
filtered_words

всего русских стоп-слов 151


['Эти', 'типы', 'стали', 'цеху']

Ту же самую задачу в других библиотеках можно решить при помощи фильтра частей речи. Можно считать, что значимыми являются лишь существительные, прилагательные, глаголы, причастия и деепричастия. Ниже приведены названия частей речи для разных библиотек.
<table>
<tr><th>Часть речи</th><th>Pymorphy</th><th>Mystem</th><th>NLTK</th></tr>
<tr><td>Существительное</td><td>NOUN</td><td>S</td><td>NN</td></tr>
<tr><td>Прилагательное</td><td>ADJF, ADJS</td><td>A</td><td>NNP</td></tr>
<tr><td>Глагол</td><td>VERB</td><td>V</td><td>JJ</td></tr>
<tr><td>Причастие</td><td>PRTF, PRTS</td><td>V</td><td>NNP</td></tr>
<tr><td>Деепричастие</td><td>GRND</td><td>V</td><td>NNP</td></tr>
</table>


Возьмем новость с сайта Лента.ру и приведем все слова её текста к начальным формам при помощи разных библиотек. Прибавим при этом к словам части речи.

In [14]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [9]:
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')

In [12]:
souped = bs(page.text)

title = souped("h1")[0].get_text()
when = souped.find_all("div", attrs={'class': 'b-topic__info'})[0]("time")[0].get_text().strip()
author = souped.find_all("span", attrs={'itemprop': 'name'})[0].get_text()
description = souped.find_all("meta", attrs={'itemprop': 'description'})[0]["content"]
text = '\n'.join([p.get_text() for p in souped.find_all("div", attrs={'itemprop': 'articleBody'})[0]("p")])

print(title)
print(when)
print(author)
print(description)
print('\n', text)

Обнаружен неожиданный эффект от употребления яблок
15:35, 27 февраля 2021
Соня Кошечкина
Содержащиеся в этих плодах фитонутриенты способствуют образованию новых нейронов

 Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.
Ранее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утв

In [24]:
# Pymorphy
def normalizePymorphy(text):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        words.append(pv[0].normal_form + '_' + str(pv[0].tag.POS)) # Берем наиболее вероятную форму.
    return words    
        
# Обратите внимание, что про иностранные слова словарь ничего не знает.
normalizePymorphy(text)

['учёный_NOUN',
 'из_PREP',
 'университет_NOUN',
 'квинсленд_NOUN',
 'и_CONJ',
 'немецкий_ADJF',
 'центр_NOUN',
 'нейродегенеративный_ADJF',
 'заболевание_NOUN',
 'обнаружить_VERB',
 'неожиданный_ADJF',
 'эффект_NOUN',
 'от_PREP',
 'употребление_NOUN',
 'яблоко_NOUN',
 'результат_NOUN',
 'исследование_NOUN',
 'появиться_VERB',
 'в_PREP',
 'научный_ADJF',
 'журнал_NOUN',
 'stem_None',
 'cell_None',
 'reports_None',
 'опыт_NOUN',
 'проводиться_VERB',
 'на_PREP',
 'мышь_NOUN',
 'специалист_NOUN',
 'культивировать_VERB',
 'стволовой_ADJF',
 'клетка_NOUN',
 'мозг_NOUN',
 'взрослый_NOUN',
 'мышей_NOUN',
 'и_CONJ',
 'добавлять_VERB',
 'в_PREP',
 'они_NPRO',
 'содержаться_PRTF',
 'в_PREP',
 'яблоко_NOUN',
 'фитонутриент_NOUN',
 'исследование_NOUN',
 'показать_VERB',
 'что_CONJ',
 'высокий_ADJF',
 'концентрация_NOUN',
 'фитонутриент_NOUN',
 'способствовать_VERB',
 'образование_NOUN',
 'новый_ADJF',
 'нейрон_NOUN',
 'по_PREP',
 'слово_NOUN',
 'учёный_ADJF',
 'определённый_ADJF',
 'фитонутриент_N

In [26]:
# PyMystem
def normalizePymystem(text):
    tokens = mystem.analyze(text)
    words = []
    for t in tokens:
        if 'analysis' in t.keys():
            if t['analysis'] != []:
                words.append(t['analysis'][0]['lex']+'_'+t['analysis'][0]['gr'][0])
            else:
                words.append(t['text']+'_'+'U')
    return words    
        
# Не все считают, что причастие всегда выступает в роли глагола, но иногда так значительно проще.
normalizePymystem(text)

['ученый_S',
 'из_P',
 'университет_S',
 'квинсленд_S',
 'и_C',
 'немецкий_A',
 'центр_S',
 'нейродегенеративный_A',
 'заболевание_S',
 'обнаруживать_V',
 'неожиданный_A',
 'эффект_S',
 'от_P',
 'употребление_S',
 'яблоко_S',
 'результат_S',
 'исследование_S',
 'появляться_V',
 'в_P',
 'научный_A',
 'журнал_S',
 'Stem_U',
 'Cell_U',
 'Reports_U',
 'опыт_S',
 'проводиться_V',
 'на_P',
 'мышь_S',
 'специалист_S',
 'культивировать_V',
 'стволовой_A',
 'клетка_S',
 'мозг_S',
 'взрослый_S',
 'мышь_S',
 'и_C',
 'добавлять_V',
 'в_P',
 'они_S',
 'содержаться_V',
 'в_P',
 'яблоко_S',
 'фитонутриент_S',
 'исследование_S',
 'показывать_V',
 'что_C',
 'высокий_A',
 'концентрация_S',
 'фитонутриент_S',
 'способствовать_V',
 'образование_S',
 'новый_A',
 'нейрон_S',
 'по_P',
 'слово_S',
 'ученый_S',
 'определенный_A',
 'фитонутриент_S',
 'положительно_A',
 'влиять_V',
 'на_P',
 'работа_S',
 'орган_S',
 'в_P',
 'тот_A',
 'число_S',
 'мозг_S',
 'выясняться_V',
 'что_C',
 'они_S',
 'оказывать_V',
 'на

In [27]:
# NLTK
def normalizeNLTK(text):
    tokens = nltk.pos_tag(nltk.word_tokenize(text))
    words = []
    for t in tokens:
        if t[0] != t[1]:
            words.append(t[0]+'_'+t[1])
    return words    
        
# А вот здесь с частеречной разметкой всё плохо, а параметров нет вовсе.
normalizeNLTK(text)

['Ученые_JJ',
 'из_NNP',
 'Университета_NNP',
 'Квинсленда_NNP',
 'и_NNP',
 'Немецкого_NNP',
 'центра_NNP',
 'нейродегенеративных_NNP',
 'заболеваний_NNP',
 'обнаружили_NNP',
 'неожиданный_NNP',
 'эффект_NNP',
 'от_NNP',
 'употребления_NNP',
 'яблок_NNP',
 'Результаты_VB',
 'исследования_JJ',
 'появились_NNP',
 'в_NNP',
 'научном_NNP',
 'журнале_NNP',
 'Stem_NNP',
 'Cell_NNP',
 'Reports_NNP',
 'Опыты_VB',
 'проводились_JJ',
 'на_NNP',
 'мышах_NNP',
 'Специалисты_VB',
 'культивировали_JJ',
 'стволовые_NNP',
 'клетки_NNP',
 'мозга_NNP',
 'взрослых_NNP',
 'мышей_NNP',
 'и_NNP',
 'добавляли_NNP',
 'в_NNP',
 'них_NNP',
 'содержащиеся_NNP',
 'в_NNP',
 'яблоках_NNP',
 'фитонутриенты_NNP',
 'Исследование_NN',
 'показало_NN',
 'что_NNP',
 'высокая_NNP',
 'концентрация_NNP',
 'фитонутриентов_NNP',
 'способствует_NNP',
 'образованию_NNP',
 'новых_NNP',
 'нейронов_NNP',
 'По_VB',
 'словам_JJ',
 'ученых_NNP',
 'определенные_NNP',
 'фитонутриенты_NNP',
 'положительно_NNP',
 'влияют_NNP',
 'на_NNP',


In [28]:
from collections import Counter # Не считать же частоты самим.

In [31]:
words = normalizePymorphy(text)
wdict = Counter(words) # Объект сразу посчитает частоты элементов списка.
print(wdict)
print()
print({w:n for w,n in wdict.items() if n>1}) # Посмотрим какие слова встречаются больше одного раза.

Counter({'в_PREP': 5, 'что_CONJ': 5, 'и_CONJ': 4, 'яблоко_NOUN': 4, 'на_PREP': 3, 'фитонутриент_NOUN': 3, 'учёный_NOUN': 2, 'из_PREP': 2, 'университет_NOUN': 2, 'эффект_NOUN': 2, 'исследование_NOUN': 2, 'мозг_NOUN': 2, 'они_NPRO': 2, 'тот_ADJF': 2, 'квинсленд_NOUN': 1, 'немецкий_ADJF': 1, 'центр_NOUN': 1, 'нейродегенеративный_ADJF': 1, 'заболевание_NOUN': 1, 'обнаружить_VERB': 1, 'неожиданный_ADJF': 1, 'от_PREP': 1, 'употребление_NOUN': 1, 'результат_NOUN': 1, 'появиться_VERB': 1, 'научный_ADJF': 1, 'журнал_NOUN': 1, 'stem_None': 1, 'cell_None': 1, 'reports_None': 1, 'опыт_NOUN': 1, 'проводиться_VERB': 1, 'мышь_NOUN': 1, 'специалист_NOUN': 1, 'культивировать_VERB': 1, 'стволовой_ADJF': 1, 'клетка_NOUN': 1, 'взрослый_NOUN': 1, 'мышей_NOUN': 1, 'добавлять_VERB': 1, 'содержаться_PRTF': 1, 'показать_VERB': 1, 'высокий_ADJF': 1, 'концентрация_NOUN': 1, 'способствовать_VERB': 1, 'образование_NOUN': 1, 'новый_ADJF': 1, 'нейрон_NOUN': 1, 'по_PREP': 1, 'слово_NOUN': 1, 'учёный_ADJF': 1, 'опреде

Фактически, выше мы провели преобразование текста в вектор. Пространство вектора определено на словаре текста - количество измерений совпадает с количеством слов, каждому измерению сопоставлено какое-то слово и отложена его частота. Подобный подход называют мешком слов (Bag of Words, BoW), так как все слова перемешиваются, их порядок больше не соблюдается, а сами слова сваливаются в один "мешок".

![](img/donkey_carrot_text.png)

Обратите внимание на распределение частот в отдельных словах и парах. Такое распределение называется [распределением Ципфа](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%A6%D0%B8%D0%BF%D1%84%D0%B0) и является характерным практически для любого распределения частот слов и их комбинаций в текстах на любом естественном языке.

$p_i=\frac{p_0}{\beta^{-\alpha*i}}$, где $\alpha\approx1$.

![](https://upload.wikimedia.org/wikipedia/ru/thumb/d/d8/WikipediaZipf20061023.png/450px-WikipediaZipf20061023.png)

Но вообще, для расчета частот существует CountVectorizer, который позволяет сделать это всё за один раз и очень хорошо ложится в конвейер.

Кстати, всё то же самое можно было посчитать при помощи класса [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) из библиотеки [skLearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text). Но её мы оставим на самостоятельное изучение.

А теперь обратите внимание на ещё пару фактов. Во-первых, наиболее частотными оказали служебные слова. Во-вторых, не все слова из аннотации, написанной человеком, вошли в список наиболее частотных.

In [33]:
print({w:n for w,n in wdict.items() if n>1})
description

{'учёный_NOUN': 2, 'из_PREP': 2, 'университет_NOUN': 2, 'и_CONJ': 4, 'эффект_NOUN': 2, 'яблоко_NOUN': 4, 'исследование_NOUN': 2, 'в_PREP': 5, 'на_PREP': 3, 'мозг_NOUN': 2, 'они_NPRO': 2, 'фитонутриент_NOUN': 3, 'что_CONJ': 5, 'тот_ADJF': 2}


'Содержащиеся в этих плодах фитонутриенты способствуют образованию новых нейронов'

Напишем функцию, которая бы оставляла слова только со значимыми частями речи (краткое и полное прилагательное, существительное, глагол, краткая и полная формы причастия, деепричастие).

In [38]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND']

def getMostFrequentWordsFiltered(text):
    
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in imp_POS and pv[0].normal_form != 'быть':
            words.append(pv[0].normal_form)
    wdict = Counter(words)
    wdict = {w:n for w,n in wdict.items() if n>1}
    return sorted(wdict.items(), key = lambda x: x[1], reverse=True)

In [39]:
getMostFrequentWordsFiltered(text)

[('яблоко', 4),
 ('учёный', 3),
 ('фитонутриент', 3),
 ('университет', 2),
 ('эффект', 2),
 ('исследование', 2),
 ('мозг', 2),
 ('тот', 2)]

Представим себе, что мы взяли много текстов и для каждого из них посчитали вектор слов. Результатом работы является [разреженная матрица](https://docs.scipy.org/doc/scipy/reference/sparse.html) частот слов. Если мы возьмем большое количество текстов, то в кажом из них встречается не так много разных слов, но словарь всех текстов вместе будет огромен. Обработка текстов должна вестись в едином пространстве. Пусть это будет пространство словаря всех текстов (в противном случае у каждого текста будет свое собственное пространство, что чрезвычайно неудобно). Получается, что для текста с маленьким словарем мы должны хранить большое число нулей. Для того, чтобы этого избежать, хранят, например, один раз номер строки, индексы ненулевых значений и сами значения, то есть чуть больше двух чисел на ненулевое значение. Если считать, что словарь заметки - 100 слов, а словарь всех текстов - 100 000 слов, мы получаем экономию места в 500 раз. То, что считалось на кластере, теперь может считаться на недобуке с 2 Гб оперативной памяти.

![](img/term-document-matrix-bow-annotated.png)
Изображение взято [отсюда](https://livebook.manning.com/book/natural-language-processing-in-action/chapter-4/v-4/61).

Теперь попробуем другой показатель для подсчета важности слов в тексте - $TF*IDF$. Здесь $TF$ - Term Frequency, частота термина в документе, а $IDF$ - Inverted Document Frequency, обратная частота термина в коллекции (количество документов, в которых встречается данный термин).

Идея метрики очень проста. Если слово встречается почти во всех документах - его различительная сила очень мала и само слово не является важным. Если слово часто встречается в данном документе, то оно являетсяя важным для него.


![](img/term-document-matrix-tfidf-annotated.png)
Изображение взято [отсюда](https://livebook.manning.com/book/natural-language-processing-in-action/chapter-4/v-4/61).


Метрика считается на коллекции документов для каждого слова, каждого документа. Для расчета меры можно использовать `TfidfVectorizer`, который работает так же как `CountVectorizer`.

Но нам необходимо искать новости, которые интересны пользователю.

Для определения меры сходства двух статей теперь может использоваться косинусная мера сходства, рассчитываемая по следующей формуле: $cos(a,b)=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$.

Косинусная мера сходство смотрит не на близость конечных точек, а на то, в какую сторону направлены векторы.

![](img/cosine_watch.jpg)

Вообще-то, использовать стандартную функцию рассчета косинусной меры сходства из <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html">sklearn</a> было бы быстрее. Но в данной задаче нам бы пришлось сводить все словари в один, чтобы на одних и тех же местах в векторе были частоты одних и тех же слов. Чтобы избежать подобной работы, напишем собственную функцию рассчета косинусного расстояния, работающую с разреженными векторами в виде питоновских словарей.

Здесь следует остановиться на том, что косинусная мера является мерой расстояния между объектами, причем подобных мер существует достаточно много. Обычно мы используем Евклидово расстояние: $d_E=\sqrt{\sum {(a_i-b_i)^2}}$. Но при обработке текстов оно работает гораздо хуже. Представим себе, что у нас есть два текста: текст статьи и текст статьи, объединенный с самим собой. Евклидово расстояние между ними будет значительным, тогда как содержание не изменится.

Помимо Евклидового расстояния часто используется Манхэттенское расстояние (расстояние городских кварталов). Если взглянуть на карту Манхэттена, то мы увидим, что практически все улицы параллельны и перпендикулярны друг другу. Это означает, что выбирая любой не удлинняющий маршрут из одной точки в другую я пройду одно и то же расстояние: $d_M=\sum {|a_i-b_i|}$. Манхэттенское расстояние используется в тех случаях, когда мы берем, например, взвешенную сумму параметров с тем, чтобы получить единую оценку. Например, оценивая различные офисы мы складываем с некоторыми коэффициентами стоимость, площадь, расстояние от центра или дома, оценку инфраструктуры и так далее. Аналогично можно брать разницу между двумя векторными представлениями офисов, чтобы найти насколько они сходны.

Расстояние Жаккарда берет отношение размера пересечения словарей к их объединению: $d_J=\frac{A \cup B}{A \cap B}$. Эта мера проверяет степень совпадения словарей двух текстов. Если словари совпадают полностью, то тексты, скорее всего, говорят об одном и том же.

Однако, в одном тексте может обсуждаться производство шестеренок, а во вводной части однажды будет упомянуто, что они необходимы для сбора механизмов, тогда как в другом тексте будут обсуждаться сами механизмы с единственным упоминанием, что они состоят из шестеренок. Случайное появление отдельных слов сделает тексты более похожими, чем это следует из их содержания. Этот недостаток устраняет косинусная мера сходства: $d_{cos}=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$. Если в одном из текстов не встречается слово из другого текста, то соответствующий член суммы в числителе будет равен нулю. Если в одном тексте слово встречается часто, а в другом редко, произведение не будет слишком большим. Проблему представляет случай, когда в обоих текстах есть несколько очень часто встречающихся слов. Тогда их произведение будет забивать все остальные слова, искажая общий смысл.

Следует иметь в виду, что косинусная мера сходства является величиной, обратно зависящей от расстояния: два одинаковых текста будут иметь косинусное сходство равное 1, тогда как расстояние между ними равно нулю, и наоборот.

В качестве мер расстояния также используются [корреляция](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D1%8F) и [дивергенция Кулльбака-Лейблера](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9A%D1%83%D0%BB%D1%8C%D0%B1%D0%B0%D0%BA%D0%B0_%E2%80%94_%D0%9B%D0%B5%D0%B9%D0%B1%D0%BB%D0%B5%D1%80%D0%B0).
